In [1]:
# Si vous êtes sur Kaggle ou Colab, exécutez cette cellule en premier
!pip install -q sentence-transformers faiss-cpu transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 97.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.5 MB/s eta 0:00:0000:0100:01
ERROR: pip's de

In [2]:
import numpy as np
import pickle
import faiss
import torch
import re
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from collections import defaultdict
import os

# Vérification du GPU
device = 0 if torch.cuda.is_available() else -1
print(f"⚙️ Utilisation du device : {'GPU 🚀' if device == 0 else 'CPU 🐢'}")

2025-12-14 15:13:11.609874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765725191.799515      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765725191.858607      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

⚙️ Utilisation du device : GPU 🚀


In [3]:
print("🔄 Chargement des fichiers...")

# Assurez-vous que les chemins sont corrects
EMBEDDINGS_PATH = "/kaggle/input/politixpert-rag-data/embeddings.npy"
METADATA_PATH = "/kaggle/input/politixpert-rag-data/metadata.pkl"

if os.path.exists(EMBEDDINGS_PATH) and os.path.exists(METADATA_PATH):
    # 1. Charger les vecteurs
    embeddings = np.load(EMBEDDINGS_PATH).astype("float32")
    
    # 2. Charger les métadonnées
    with open(METADATA_PATH, 'rb') as f:
        metadata = pickle.load(f)
        
    # 3. Extraire les textes
    chunks = [m['text'] for m in metadata]
    
    print(f"✅ Données chargées : {len(chunks)} documents.")
    
    # 4. Création de l'index FAISS
    print("🗂️ Construction de l'index FAISS...")
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    print("✅ Index prêt !")
else:
    raise FileNotFoundError("❌ ERREUR : Les fichiers 'embeddings.npy' ou 'metadata.pkl' sont introuvables. Veuillez les importer.")

🔄 Chargement des fichiers...
✅ Données chargées : 90727 documents.
🗂️ Construction de l'index FAISS...
✅ Index prêt !


In [4]:
print("🧠 Chargement des modèles IA...")

# Modèle d'embedding (Pour la question uniquement)
embed_model = SentenceTransformer("intfloat/multilingual-e5-base", device="cuda" if device == 0 else "cpu")

# Modèle de Génération (Qwen 1.5B Instruct)
gen_model_id = "Qwen/Qwen2.5-1.5B-Instruct"

generator = pipeline(
    "text-generation",
    model=gen_model_id,
    device=device,
    torch_dtype=torch.float16 if device == 0 else torch.float32,
    trust_remote_code=True
)
print("✅ Modèles chargés avec succès !")

🧠 Chargement des modèles IA...


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


✅ Modèles chargés avec succès !


In [5]:

def clean_output(text):
    """Supprime les caractères chinois et nettoie le texte"""
    text = re.sub(r'[\u4e00-\u9fff]+', '', text) # Supprime Hanzi
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def semantic_search(query, top_k=20, min_score=0.82):
    """Recherche avec filtrage intelligent"""
    # Encodage
    q_emb = embed_model.encode([query], normalize_embeddings=True).astype("float32")
    
    # Recherche large
    search_k = top_k * 2
    scores, indices = index.search(q_emb, search_k)
    
    results = []
    
    for i in range(search_k):
        score = float(scores[0][i])
        idx = indices[0][i]
        
        # Filtre 1 : Score (Pertinence)
        if score < min_score:
            continue
            
        # Filtre 2 : Longueur (Qualité)
        if idx < len(chunks):
            text_content = chunks[idx]
            if len(text_content) < 50:
                continue
                
            # Ajout du résultat
            meta = metadata[idx]
            results.append({
                "score": score,
                "text": text_content,
                "title": meta.get("title", "Sans titre"),
                "description": meta.get("description", ""),
                "date": meta.get("date", ""),
                "source": meta["source"],
                "link": meta.get("link", "#")
            })
            
            if len(results) >= top_k:
                break
                
    return results

def group_results_by_party(results):
    grouped = defaultdict(list)
    for r in results:
        grouped[r["source"]].append(r)
    return grouped

def build_context_for_party(docs, max_docs=4):
    context = []
    sources = []
    for d in docs[:max_docs]:
        # Concaténation Titre + Description + Texte
        doc_entry = f"""
                    معلومات الوثيقة:
                    - التاريخ: {d['date']}
                    - العنوان: {d['title']}
                    - السياق: {d['description']}
                    - النص: {d['text']}
                    """
        context.append(doc_entry)
        sources.append({"title": d['title'], "date": d['date']})
    
    return "\n___________________\n".join(context), sources

In [6]:

def generate_party_summary(question, party, docs):
    context, sources = build_context_for_party(docs)

    # Prompt "Blindé" (Robust)
    messages = [
        {"role": "system", "content": "أنت محلل سياسي خبير ومحايد. اكتب باللغة العربية الفصحى فقط."},
        {"role": "user", "content": f"""
استناداً للوثائق التالية، لخص موقف حزب "{party}" بخصوص السؤال: "{question}".

الوثائق المتاحة:
{context}

⚠️ تعليمات صارمة (Strict Instructions):
1. **اللغة**: اكتب باللغة العربية فقط (Arabic Only). لا تستخدم أحرف صينية أو رموز.
2. **التواريخ**: انتبه لتواريخ الوثائق لفهم السياق (مثلاً نصوص نهاية 2023 تتحدث عن مالية 2024).
3. **الفضاء**: إذا كان السؤال عن "غزو الفضاء" أو "الكواكب" والنصوص سياسية، اكتب "لا توجد معلومات".
4. **التلخيص**: لخص الموقف في فقرة أو فقرتين بصيغة الغائب (يرى الحزب، يؤكد الحزب).
"""}
    ]

    try:
        output = generator(
            messages,
            max_new_tokens=300,
            do_sample=False,
            return_full_text=False
        )
        raw_text = output[0]["generated_text"]
        summary = clean_output(raw_text) # Nettoyage final
    except Exception as e:
        summary = "حدث خطأ أثناء التلخيص."
        print(f"Error: {e}")

    return {
        "party": party,
        "summary": summary,
        "sources": sources
    }

def rag_answer(question, top_k=20):
    print(f"🔍 Recherche pour : '{question}'...")
    results = semantic_search(question, top_k=top_k)
    
    if not results:
        return []
        
    grouped = group_results_by_party(results)
    answers = []
    
    print(f"📊 Analyse de {len(grouped)} partis...")
    for party, docs in grouped.items():
        ans = generate_party_summary(question, party, docs)
        answers.append(ans)
        
    return answers

In [7]:

# Choisissez votre question ici
question = "ما هي مقترحات الأحزاب لإصلاح التعليم؟"

# Exécution
final_answers = rag_answer(question, top_k=20)

# Affichage Markdown
print("\n" + "="*60 + "\n")

if not final_answers:
    print("❌ لم يتم العثور على وثائق ذات صلة (تمت تصفية النتائج غير الدقيقة).")
else:
    for a in final_answers:
        print(f"### 🏛 الحزب: {a['party']}")
        print("**📄 الخلاصة:**")
        print(a['summary'])
        print("\n**🔗 المصادر:**")
        for s in a['sources']:
            d = s['date'] if s['date'] and s['date'] != 'nan' else "تاريخ غير متوفر"
            print(f"- {s['title']} ({d})")
        print("-" * 50)

🔍 Recherche pour : 'ما هي مقترحات الأحزاب لإصلاح التعليم؟'...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


📊 Analyse de 3 partis...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.




### 🏛 الحزب: التجمع الوطني للأحرار
**📄 الخلاصة:**
**موقف حزب التجمع الوطني للأحرار بشأن إصلاح التعليم** حزب التجمع الوطني للأحرار يرى أن الحلول المقترحة لمشكلة التعليم تتضمن عدة نقاط رئيسية. أولًا، يعتقد الحزب أن مشاركة الشباب في العملية السياسية وتمكينهم من فرص عمل تمكنهم من تحقيق الثقة في النظام السياسي يشكل أساسًا مهمًا لتحقيق ذلك. ثانيًا، يقترح الحزب تقديم حلول عملية وقابلة للتنفيذ للنهوض بالتعليم، بما في ذلك دمج الأطفال ذوي الاحتياجات الخاصة وتطوير المناهج التعليمية. ثالثًا، يشير الحزب إلى أهمية تحسين ظروف العمل للأساتذة والطلاب، خاصة في المناطق القروية، حيث يقترح توحيد المدارس الجماعية وإعادة النظر في نظام التعليم المهني والتدريب المهني. رابعًا، يؤكد الحزب على ضرورة تعاون الحزب مع السلطات المحلية والجماعات المحلية لتنفيذ هذه المبادرات، مع التركيز على تحسين ظروف العمل للأساتذة والطلاب. في الختام، يؤكد الحزب على أهمية دور الأساتذة والأساتذة في بناء المنظومة التعليمية، وأن عدم وجود أساتذة ملتزمين ومؤهلين لن يؤدي إلى تقدم التعليم.

**🔗 المصادر:**
- السعدي مفتاح الكرامه واسترجاع ثقه